<a href="https://colab.research.google.com/github/HoodBlack/CNN_SKKU/blob/main/preTrainedViT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained ViT

In [ ]:
pip install timm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import timm
import matplotlib.pyplot as plt
from torch.optim import Optimizer
import torch.nn as nn
from torch.optim import lr_scheduler
import time
import copy

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [ ]:
pip install torch_optimizer

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from PIL import Image
from skimage import io
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision import models
import torchvision.transforms as transforms

In [ ]:
import json
import csv

def json_to_csv(folder_path, output_file):
    # CSV 파일을 작성하기 위한 파일 핸들러를 엽니다.
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        writer.writerow(['file_name','grade'])
        # 폴더 내의 모든 파일에 대해 반복합니다.
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):  # JSON 파일인지 확인합니다.
                file_path = os.path.join(folder_path, filename)

                # JSON 파일을 읽어옵니다.
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                    file_name = data.get('label_info', {}).get('image', {}).get('file_name', '')
                    grade = data.get('label_info', {}).get('shapes', [{}])[0].get('grade', '')
                    writer.writerow([file_name, grade])


In [ ]:
resize =224
transformation=transforms.Compose([transforms.Resize([resize, resize]),transforms.ToTensor(),])

In [ ]:
class MeatDataset(Dataset):

  def __init__(self, csv_file, root_dir, phase,transform =transformation):
    self.meat_data = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.meat_data)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir, self.meat_data.iloc[idx,0])
    img = Image.open(img_name)
    transformed_img = self.transform(img)
    label = self.meat_data.iloc[idx,1].astype(int) -1

    return transformed_img, label


In [ ]:


# 사용 예시
folder_path_train = './meat_data/train_cows_labels'  # JSON 파일들이 있는 폴더 경로로 변경해야 합니다.
output_file_train = './meat_data/train_cows/meat_train.csv' # 결과를 저장할 CSV 파일 경로로 변경해야 합니다.

folder_path_valid='./meat_data/valid_cows_labels'
output_file_valid='./meat_data/valid_cows_images/meat_valid.csv'
json_to_csv(folder_path_train, output_file_train)

json_to_csv(folder_path_valid, output_file_valid)


In [ ]:
train_csv = output_file_train
train_root = './meat_data/train_cows/'
valid_csv = output_file_valid
valid_root='./meat_data/valid_cows_images'

train_dataset = MeatDataset(csv_file=train_csv, root_dir=train_root, phase='train',transform=transformation)
valid_dataset = MeatDataset(csv_file=valid_csv, root_dir = valid_root, phase ='valid', transform = transformation)


In [ ]:
image_datasets = {'train' : train_dataset, 'valid' : valid_dataset}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}

In [ ]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
batch_size =32

train_Dataloader  = DataLoader(train_dataset, batch_size = batch_size, shuffle =False, num_workers=4)

val_Dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False, num_workers=4)

dataLoaders_dict = {"train" : train_Dataloader, "valid": val_Dataloader}

# Data 확인

In [ ]:
train=pd.read_csv(output_file_train)

In [ ]:
train.iloc[0,1].astype(int)

2

In [ ]:
valid = pd.read_csv(output_file_valid)
valid[valid['grade'].isnull()]

,file_name,grade


# Models

In [ ]:
pretrained_model_list = timm.list_models(pretrained=True)

print(pretrained_model_list)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

In [ ]:
pip install --upgrade huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


num_classes =0?

* https://huggingface.co/timm/vit_base_patch32_clip_448.laion2b_ft_in12k_in1k



In [ ]:
num_classes = 2
model = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k',pretrained=True,num_classes = num_classes)

In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

citerion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

In [ ]:
criterion = nn.CrossEntropyLoss()
scheduler = exp_lr_scheduler

In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7f0b1f40ca50>


In [ ]:
for param in model.parameters():
    print(param.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# Device configuration
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)
    print("GPU가 사용 가능합니다.")
else:
    device = torch.device("cpu")
    print("GPU를 사용할 수 없습니다.")

GPU가 사용 가능합니다.


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=100):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        print(optimizer.param_groups[0]['lr'])

        # 각 에폭(epoch)은 학습 단계와 검증 단계를 갖습니다.
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0

            # 데이터를 반복
            for inputs, labels in dataLoaders_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                # 순전파
                # 학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 통계
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 모델을 깊은 복사(deep copy)함
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # 가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer,exp_lr_scheduler,
                       num_epochs=40)

Epoch 0/39
----------
0.01


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


train Loss: 2.6420 Acc: 0.5165
valid Loss: 0.7975 Acc: 0.5000

Epoch 1/39
----------
0.01
train Loss: 0.9889 Acc: 0.5030
valid Loss: 0.7104 Acc: 0.5000

Epoch 2/39
----------
0.01
train Loss: 0.9292 Acc: 0.5000
valid Loss: 0.7498 Acc: 0.5000

Epoch 3/39
----------
0.01
train Loss: 0.8593 Acc: 0.5090
valid Loss: 0.8501 Acc: 0.5000

Epoch 4/39
----------
0.01
train Loss: 0.8020 Acc: 0.4980
valid Loss: 0.6949 Acc: 0.5000

Epoch 5/39
----------
0.01
train Loss: 0.8150 Acc: 0.4950
valid Loss: 0.7102 Acc: 0.5000

Epoch 6/39
----------
0.01
train Loss: 0.7572 Acc: 0.4860
valid Loss: 0.7585 Acc: 0.5000

Epoch 7/39
----------
0.01
train Loss: 0.7258 Acc: 0.5040
valid Loss: 0.7024 Acc: 0.5000

Epoch 8/39
----------
0.001
train Loss: 0.6975 Acc: 0.4940
valid Loss: 0.6930 Acc: 0.5000

Epoch 9/39
----------
0.001
train Loss: 0.6966 Acc: 0.5010
valid Loss: 0.6928 Acc: 0.5400

Epoch 10/39
----------
0.001
train Loss: 0.6965 Acc: 0.4880
valid Loss: 0.6926 Acc: 0.5530

Epoch 11/39
----------
0.001
trai

In [ ]:
dataset_sizes['train']

2000

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataLoaders_dict['valid']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                plt.imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)